In [32]:
import pdfplumber
text=[]
with pdfplumber.open('joker_new_final.pdf') as f :
    pages=f.pages
    for page in range(2,104):
        text.append(pages[page].extract_text(x_tolerance=3, y_tolerance=3, layout=True, x_density=7.25, y_density=13))
        
    f.close()

In [13]:
import pandas as pd
import re

In [23]:
def get_characters(page):
    characters=[]
    for character in [e.strip() for e in page if e.isupper()]:
        if character not in TRANSITIONS and character.split()[0] not in INT_EXT and len(character)<35:
            # characters.add(character)
            characters.append(character)
    return (characters )       
    # return (characters )       

In [22]:
re.sub(r"[\(\)].+$",'','DET. BURKE (OS)').strip()

'DET. BURKE'

In [15]:
def get_scene(page):
    scenes=[]
    for i in [e.strip() for e in page if e.isupper() and e not in TRANSITIONS]:
        for item in i.split():
            if item in INT_EXT:
                scene=' '.join([x for x in i.split()])
                scenes.append(scene)
                
    return scenes 

In [13]:
def get_action(page):
    act=[]
    action=[]
    
    for i in [e.split() for e in page]:
        if ' '.join(i) not in TRANSITIONS:
            action.append(' '.join(i))
            
    # for a in range(len(get_scene(page))-1):
    #     start_ind=action.index(get_scene(page)[a])+2
    #     end_ind=action.index(get_scene(page)[a+1])-2
    #     act.append([d for d in action[start_ind:end_ind] if len(d)>35])
    return action
    # return [' '.join(d) for d in [a for a in act]]

action=get_action(text)  
action

['',
 '',
 '',
 '',
 '',
 '',
 'HEAR LAUGHTER.',
 '',
 'The sound of a man totally cracking up.',
 '',
 '',
 '',
 'INT. DEPT. OF HEALTH, OFFICE - MORNING',
 '',
 "CLOSE ON ARTHUR (30's), tears in his eyes from laughing so",
 "hard. He's trying to get it under control. His greasy, black",
 "hair hanging down over his forehead. He's wearing an old,",
 'faded green cardigan sweater, a threadbare gray scarf, thin',
 'from years of use, hangs loosely around his neck.',
 "He's sitting across from an overworked SOCIAL WORKER (50's),",
 'African American. Her office is cramped and run-down in a',
 'cramped and run-down building. Stacks of folders piled high',
 'in front of her.',
 '',
 'She just sits behind her desk, waiting for his laughing fit',
 "to end, she's been through this before. Finally it subsides.",
 '',
 "Arthur takes a deep breath, pauses to see if it's over.",
 'Beat.',
 '',
 'ARTHUR',
 '--is it just me, or is it getting',
 'crazier out there?',
 '',
 "Despite the laughter, ther

In [11]:
def get_dialog(page):
    act=[]
    dialog=[]
    
    for i in [e.split() for e in page if not e.isspace()]:
        if ' '.join(i) not in TRANSITIONS:
            dialog.append(' '.join(i))
            
    for a in range(len(get_characters(page))+1):
        start_ind=dialog.index(get_dialog(page)[a])+2
        end_ind=dialog.index(get_scene(page)[a+1])-2
        act.append([d for d in dialog[start_ind:end_ind] if len(d)<35])
        
    return [' '.join(d) for d in [a for a in act]]

get_dialog(text)  

NameError: name 'get_characters' is not defined

## dialog extraction

In [ ]:
'''
for each scene 
-strip the white space and only include lines with not more than 40 letters
-find indecies of lines of where each charachter appear
-use those indecies to extract the data inbetween characters

- need to make more find a way to accomodate
'''
def dialogs(scene,characte_list):
  inx=[i for s in scene for i,line in enumerate(s) if len(line)<35 and line in sum(characte_list,[])]
  
  para=[]
  
  for i in range(len(inx)):
    if inx[i]==inx[-1]:
      para.append(sum(scene,[])[inx[i]:])
    else:
      para.append(sum(scene,[])[inx[i]:inx[i+1]])
  return para
  # return inx
dialogs(sc.get_scenes(),sc.get_characters())

NameError: name 'scene' is not defined

In [737]:
from collections import Counter

In [86]:
class Screenplay():
    
    INT_EXT = ['INT.', 'EXT.', 'INT/EXT.', 'EXT/INT.']
    TOD = ['AFTERNOON', 'CONTINUOUS', 'DAWN', 'DAY', 'DUSK',
           'EVENING', 'LATER', 'MORNING', 'NIGHT', 'NOON', 'SAME','SUNSET']
    TRANSITIONS = ['BACK TO:', 'CUT TO:', 'DISSOLVE TO:', 'FADE IN:', 'CUT BACK TO:','CUT BACK TO:',
                   'FADE OUT:', 'FADE TO:', 'JUMP CUT TO:', 'MATCH CUT TO:', 'TITLE:', 'QUICK CUTS:',
                   'SMASH CUT TO:', 'TIME CUT:', 'WIPE TO:', 'CROSSFADE:', 'OVER BLACK:', 'CUT TO BLACK',
                   'FADE TO:BLACK', 'FLASH CUT:', 'LAP DISSOLVE:', 'MATCH DISSOLVE TO:', 'SUPER:', 'IRIS OUT:']
    SHOTS = ['AERIAL SHOT', 'ANGLE ON',  'CLOSE ON', 'CLOSER ANGLE', 'CONTINUOUS',
             'CONTRAZOOM', 'CRAWL',  'ESTABLISHING SHOT',
             'EXTREMELY LONG SHOT (XLS):', 'FAVOR ON','HEAR LAUGHTER.',
             'FLASHBACK:', 'FREEZE FRAME:', 'INSERT','DAYS LATER'
             'INTO FRAME:', 'INTO VIEW:', 'MOS', 'O.C. / O.S.',
             'POV', 'PULL BACK:', 'PULL FOCUS:', 'PUSH IN:',
             'REVERSE ANGLE',  'SPLIT SCREEN SHOT:', 'STOCK SHOT:',
             'TIGHT ON', 'TIME CUT', 'V.O.', 'ZOOM:']

    def __init__(self, text: list) -> list:
        self.no_space_text = [l.strip() for l in text if not l.isspace()]
        self.text = [l.strip() for l in text]
        self.location=self.locations()
        # self.scene_index=self.separate(self.text)
        self.scenes=self.separate_by_scene_index(self.no_space_text)
        self.characters=self.get_characters()
        self.dia_text=self.get_dialog_text()
        # self.dialog=self.dialogs()
        
        
        

    def locations(self):
        locations = []
        for i in [e.strip() for e in self.text if e.isupper() and e not in self.TRANSITIONS]:
            for item in i.split():
                if item in self.INT_EXT:
                    scene = ' '.join([x for x in i.split()])
                    locations.append(scene)
        return locations
    
    # def get_scene_index(self):
    #     return [i for i,l in enumerate([e.strip() for e in self.original_text]) if l in self.location]
    
###########################

    def separate_by_scene_index(self,text):
        scene_index=[i for i,l in enumerate([e for e in text]) if l in self.location]
        result=[]
        for i,j in enumerate(scene_index):
            if scene_index[i]==scene_index[-1]:
                result.append(text[scene_index[i]:])
            else:
               result.append(text[scene_index[i]:scene_index[i+1]])
        return result    
    
   ##################
    def get_action_desc(self):
       return [
                t for t in self.text
                if 
                t.strip() not in self.TRANSITIONS
                and t.strip() not in self.SHOTS
                and t !=''
                and len(t.strip())>36
                or t in self.location
                # and not '!' in t 
                # and not '?' in t 
                # and not t.endswith('.')
                # and not t.startswith('(')
                # or ')' in t
                ]
   
   
    def get_dialog_text(self):
        
        return [
                t for t in self.text
                if 
                # and t.strip().isupper() 
                # and len(t)<15 
                t.strip() not in self.TRANSITIONS
                and t.strip() not in self.SHOTS
                and t !=''
                and len(t.strip())<36
                or t in self.location
                and not '!' in t 
                and not '?' in t 
                and not t.endswith('.')
                and not t.startswith('(')
                or ')' in t
                ]
        
    def get_characters(self):
        return [
                t for t in self.text
                if t.isupper() 
                and t not in self.TRANSITIONS
                and t not in self.SHOTS
                and len(t.strip())<36
                and t not in self.location 
                and not '!' in t 
                and not '?' in t 
                and not t.endswith('.')
                and not t.startswith('(')
                ]
            
        
                
    def get_dia(self):
        scenes=[]
        ind=[i for i,l in enumerate(self.dia_text)if l in self.characters]
        # for j in self.scenes:
        #     [scenes.append(self.dia_text[ind[i]:ind[i+1]]) for i in range(len(ind)-1)]
             
        for j in self.scenes:
            for k in j:
                [scenes.append(self.dia_text[ind[i]:ind[i+1]]) for i in range(len(ind)-1)]
             
        return scenes
        # return ind
        # return [i for i in ind] 
        # return [(i,s) for i,s in enumerate(scenes) ] 
        # return [s for s in scenes if len(s)>1] 
       
            
    def load(self):      
        scenes_head={}
        for i,s in enumerate(self.locations(),start=1):
            
            scenes_head.update({
                i:{
                f'int_ext':[l for l in s.split()][0],
                'location':' '.join([l for l in s.split() if l not in sc.TOD and l!='-'][1:-1]),
                'tod':[l for l in s.split() if l in sc.TOD][0]
    #             # 'action': self.get_action()[i-1]
                
                }})    
    
        return scenes_head
    
    @classmethod
    def add_transition(cls,transition):
        cls.TRANSITIONS.append(transition)
    @classmethod
    def add_shot(cls,shot):
        cls.TRANSITIONS.append(shot)
        
with open('text.txt', 'r') as f:
    text = f.readlines()

sc = Screenplay(text[:2000])
# sc.get_dia()
# sc.dia_text
# sc.get_dialog_text()
# for i in sc.separate_by_scene_index(sc.no_space_text)[0]:
#     print(i)
sc.get_action_desc()
# sc.characters

['The sound of a man totally cracking up.',
 'INT. DEPT. OF HEALTH, OFFICE - MORNING',
 "CLOSE ON ARTHUR (30's), tears in his eyes from laughing so",
 "hard. He's trying to get it under control. His greasy, black",
 "hair hanging down over his forehead. He's wearing an old,",
 'faded green cardigan sweater, a threadbare gray scarf, thin',
 'from years of use, hangs loosely around his neck.',
 "He's sitting across from an overworked SOCIAL WORKER (50's),",
 'African American. Her office is cramped and run-down in a',
 'cramped and run-down building. Stacks of folders piled high',
 'She just sits behind her desk, waiting for his laughing fit',
 "to end, she's been through this before. Finally it subsides.",
 "Arthur takes a deep breath, pauses to see if it's over.",
 "Despite the laughter, there's real pain in his eyes.",
 "Something broken in him. Looks like he hasn't slept in days.",
 'He reluctantly reaches into the pocket of his jacket hanging',
 'on the chair behind him. Pulls out a

In [7]:
with open('text.txt', 'r') as f:
    text = f.readlines()

sc = Screenplay(text[:2000])

# sc.no_space_text
# sc.location
# sc.characters
# sc.scene_index
# sc.scenes
# sc.dialog
# sc.dia_text
# sc.get_dialog_text()
sc.get_dia()[17][1]
# 

# sc.load()
# sc.get_characters()

# scenes_head={}
# for k in sc.dialog:
#     # if k[1:]:
#     # print(k[0])
#         scenes_head[k[0]]=k[1:]

# for i,s in enumerate(sc.dialog,start=1):
    
#     scenes_head.update({
#         i:{
#         'dialog':{k.split() for k,v in s},
#         # 'int_ext':[l for l in s.split()][0],
#         # 'location':' '.join([l for l in s.split() if l not in sc.TOD and l!='-'][1:-1]),
#         # 'tod':[l for l in s.split() if l in sc.TOD]
# #             # 'action': self.get_action()[i-1]
#         }})
# scenes_head 
        
    

['ARTHUR',
 'Yes, Ma. Nothing. No letter.',
 '',
 'seem to notice anyway.',
 '',
 '',
 '9.',
 '',
 '']

In [689]:
text=[]
for i in sc.original_text:
    if i.isalpha and i!='\n':
        text.append(i.strip('\n'))




In [22]:
len(sc.scenes)

49

# SELECTING NON CONSECTUTIVE INDEcies

In [11]:
x = [1, 2, 4, 7, 9, 10, 45]

[n for i,n in enumerate(x,start=-1) if x[i]+1!=x[i+1]]


[{-1: 1}, {1: 4}, {2: 7}, {3: 9}, {5: 45}]

In [ ]:
for s in sc.scenes:
    for k in s:
        for c in sc.characters:
            for i in c:
                print(c)
        

In [ ]:
characters=[]
for s in sc.scenes:
    for j in s:
        for c in sc.characters:
            
            t.strip() for t in s[sc.scenes[i]:sc.scenes[i+1]]
                # t.strip() for t in s[sc.scenes[i]:sc.scenes[i+1]]
               
               
               
                # if t.strip() not in sc.location 
                # and len(t)<60 
                # and len(t)>25 
                # and t.isupper() 
                # and not '!' in t 
                # and not '?' in t 
                # and not t.strip().endswith('.')
                # ])
characters       

In [717]:
from string import punctuation

In [ ]:
punctuation

In [236]:
scenes={}
for i,s in enumerate(sc.locations(),start=1):
    
    scenes.update({
        i:{
        f'int_ext':[l for l in s.split()][0],
        'location':' '.join([l for l in s.split() if l not in sc.TOD and l!='-'][1:-1]),
        'tod':[l for l in s.split() if l in sc.TOD][0]
        
        }})

# if contiuos grab before
scenes

{1: {'int_ext': 'INT.', 'location': 'DEPT. OF HEALTH,', 'tod': 'MORNING'},
 2: {'int_ext': 'EXT.',
  'location': 'GOTHAM SQUARE, MIDTOWN',
  'tod': 'AFTERNOON'},
 3: {'int_ext': 'EXT.',
  'location': 'CORNER, ALLEY GOTHAM',
  'tod': 'CONTINUOUS'},
 4: {'int_ext': 'INT.',
  'location': 'CITY BUS (MOVING) HEADING UPTOWN',
  'tod': 'AFTERNOON'},
 5: {'int_ext': 'EXT.', 'location': 'THE BRONX,', 'tod': 'SUNSET'},
 6: {'int_ext': 'EXT.', 'location': 'SIDE ALLEY, TENEMENTS', 'tod': 'DUSK'},
 7: {'int_ext': 'EXT.', 'location': 'STEEP STAIRWAY,', 'tod': 'DUSK'},
 8: {'int_ext': 'INT.', 'location': 'APARTMENT BUILDING,', 'tod': 'DUSK'},
 9: {'int_ext': 'INT.',
  'location': 'ELEVATOR, APARTMENT',
  'tod': 'CONTINUOUS'},
 10: {'int_ext': 'INT.',
  'location': "MOM'S APARTMENT, FRONT",
  'tod': 'EVENING'},
 11: {'int_ext': 'INT.', 'location': "MOM'S BEDROOM,", 'tod': 'NIGHT'}}